### 语言模型

语言模型是一种序列模型

常见的应用就是根据一句话中前一个部分的内容（词），来预测后面的词句

序列模型的原始数据就是一个完整的很长的序列，我们需要将其整理成为可用于训练的数据（样本特征，样本标签）

在序列模型中，样本特征可以认为是前 n 个词，样本标签可以认为是第 n + 1 个词

<br>

### 马尔可夫假说

对于一个序列模型，每一个值可能只与前 n 个词相关，跟更加远久的数据关系不大

比如股价的变化，今天股票的变化可能与近两年股票变化数据相关，但是与 10 年前的数据几乎没关系了

若在一个序列模型中预测一个值只用到了前 n 个值的信息，那么称 n 为这个序列模型的时间步长

<br>

### 语言模型数据预处理实例

读取指定的语言文本数据

In [1]:
with open('dataset/article.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

<br>

词元化，指定一个 token 的单位，将文本划分成 token 组成的序列

这里将每一个词作为一个 token（有时以两个词作为一个 token，或者一个字符作为一个 token）

In [100]:
import re
lines = [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]  # 去除标点，将大写转为小写
tokens = [line.split() for line in lines]    # 对每一个词进行分割

<br>

设计词表，将字符串用数字来表示

词表其实就是 token 到 数值的映射

在这个部分，我们可以将出现次数少于某个值的词丢弃，加速训练

其中词表中一定有一个 unk 表示不知道的词（例外）

In [101]:
import collections
tokens = [token for line in tokens for token in line]      # 将 tokens 展平为一维数组
counter = collections.Counter(tokens)                      # 对 tokens 中每一个不同的词进行计数
token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True)   # 按照出现频率排序

idx_to_token = ['<unk>']      # 用映射数值来获取对应的 token 的容器，初始化 <unk> 的映射数值为 0
token_to_idx = {}             # 用token来获取对应的映射数值


min_freq = 5

# 构建两个容器
for token, freqs in token_freqs:
    if freqs < 5:
        continue
    idx_to_token.append(token)
    token_to_idx[token] = len(idx_to_token) - 1

# 这样词表就用 idx_to_token 和 token_to_idx 这两个容器保存下来了
vocab_size = len(idx_to_token)

<br>

文本数值化，应用词表，将原本的字符串文本转化为数值数组

In [ ]:
corpus = [token_to_idx.get(token, 0) for token in tokens]   # 将词表中没有的 token 统一视为 <unk>

<br>

针对整理好的数值序列进行采样

这一步需要指定好时间步长，若序列总长为 N，时间步长为 n

则可以采集 N / n 个样本，采用连续采样，初始位置可以有 N % n 的偏移量

X 即采集到的 n 个 token，对应的标签 Y 使用相对于 X 向后偏移一个 token 的序列来表示

In [103]:
import torch

time_step = 10
sample_num = len(corpus) // time_step

X = []
Y = []

# 这里直接从第一个 token 开始采样
for i in range(sample_num):
    start = i * time_step
    X.append(corpus[start : start + time_step])
    Y.append(corpus[start + 1 : start + time_step + 1])

X = torch.tensor(X)
Y = torch.tensor(Y)

# 输出前 3 个样本进行观察
for i in range(3):
    print('X{}: {}'.format(i, X[i]))
    print('Y{}: {}'.format(i, Y[i]))

X0: tensor([ 1,  0,  2, 24,  8, 12,  6,  0,  0,  1])
Y0: tensor([ 0,  2, 24,  8, 12,  6,  0,  0,  1,  0])
X1: tensor([ 0,  2,  0, 14,  0,  0,  0, 25,  4,  1])
Y1: tensor([ 2,  0, 14,  0,  0,  0, 25,  4,  1,  0])
X2: tensor([ 0,  3,  0,  1,  0,  0, 20, 21,  1,  0])
Y2: tensor([ 3,  0,  1,  0,  0, 20, 21,  1,  0,  0])


<br>

最后将数据以 one-hot 形式进行表示（独热编码）

可以理解为是一种归一化（数值只有 0 和 1）

原本数据中一个 token 使用 1 个数值表示

one-hot 后一个 token 需要用 vocab_size 个数值表示

In [ ]:
from torch.nn import functional

X = functional.one_hot(X, vocab_size)
Y = functional.one_hot(Y, vocab_size)

#  N * time_step * vocab_size
X.shape, Y.shape   

(torch.Size([95, 10, 33]), torch.Size([95, 10, 33]))

在实际训练数据时，可能会将数据的维度改为 time_step * N * vocab_size

这样做可以更方便的实现同时对多个样本进行一个时间步长上的计算（潜变量的计算）